In [ ]:
import subprocess
import pandas as pd
import os

In [ ]:
geo = 'us'  # 'us', 'eu' or 'as' only

# We read in the satelite metadata and download all necessary files
df = pd.read_csv('pm25_satellite_metadata.csv', sep=',')
df.reset_index()

active_abspath = os.path.abspath('')
train_abspath = os.path.join(active_abspath, "train")
test_abspath = os.path.join(active_abspath, "test")
# We create a folder called train/ and a folder called test/ for training and testing data respectively
if not os.path.isdir(train_abspath):
    os.mkdir(train_abspath)

if not os.path.isdir(test_abspath):
    os.mkdir(test_abspath)

# AWS version check
print("Checking AWS CLI...")
subprocess.run(["aws", "--version"], check=True, capture_output=True)
    
for idx, row in df.iterrows(): 
    filename = row['granule_id']
    download_path = row[geo + '_url']
    split = row['split']
    active_path = train_abspath if split == "train" else test_abspath
    info_string = "Current idx: {:d}, Split: {}, Filename: {}. ".format(idx, split, filename)
    if not os.path.isfile(os.path.join(active_path, filename)):
        print(info_string + "File Not Found. Downloading...")
        subprocess.run(["aws", "s3", "cp", download_path, active_path, "--no-sign-request"], check=True, capture_output=True)
    else:
        print(info_string + "File Found. Skipping...")